#**ENP 012 - Programação Linear e Inteira**

**Prova Prática**

Semestre: 2023/2

Valor: 30 Pontos

Prof. Dr. Thiago Augusto de Oliveira Silva


---

**Instruções:**


1.   Antes de começar os estudantes devem **fazer uma cópia** deste documento em seu próprio Drive;
2.   A entrega deverá ser feita **via moodle** e, para tanto, os estudantes deverão fazer o download do arquivo .pynb e enviar na plataforma;
3.   Adicionalmente os discentes deverão compartilhar o presente documento com o prof. por meio do e-mail: thiago@ufop.edu.br;
4. Não é permitido o compartilhamento deste link com outro discente durante a prova e até a entrega do resultado final pelo professor.




##Instalação e importação do pacote gurobipy

In [1]:
#instalação do gurobi
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 53.4 MB/s eta 0:00:00


In [2]:
import gurobipy as grb

# **Questão 1 (15 pontos)**

**Problema do Transporte**

O problema do transporte consiste em definir o fluxo de produtos que vai de um nó de oferta para um nó de demanda de forma a atender as demandas com custo mínimo de transporte. Ademais, a capacidade de oferta dos nós de origem devem ser respeitadas.

Seja:

*   $I$: Conjunto de nós de origem
*   $J$: Conjunto de nós de destino
*   $d_j$: Demanda do nó $j$
*   $o_i$ Oferta di  nó $i$
*   $c_{ij}$: Custo de ir de $i$ para $j$
*   $x_{ij}$ Fluxo saindo de $i$ para $j$

Considere a seguinte formulação matemática:

\begin{eqnarray}
\label{of-uflp} \mbox{Minimize} \ & Q(x,y) = \sum\limits_{i \in I} \sum\limits_{j \in J}c_{ij} x_{ij}                   & \\
\nonumber \mbox{ Sujeito a:}
        &                                   & \\
\label{r1-tp} &  \sum\limits_{i \in I} x_{ij} >= d_i   & \forall \quad j \in J \\
\label{r2-tp} &  \sum\limits_{j \in J} x_{ij} <= o_i   & \forall \quad i \in I \\
\label{r3-uflp} &  x_{ij}  \geq 0,  &\forall \quad i \in I, j \in J
\end{eqnarray}


O Código abaixo implementa o problema acima, de forma genérica em gurobi utilizando a API do Gurobipy

In [ ]:
# Instância
I = [1,2,3,4,5]
J = [1,2,3,4,5,6]
c = [ [30, 20, 24, 18, 31, 31],
      [12, 36, 30, 24, 23, 23],
      [8, 15, 25, 20, 17, 17],
     [18, 5, 15, 30, 13, 13],
     [7, 25, 15, 10, 27, 27]]

d = [5, 8, 4, 10, 16, 16]
o = [13,13,9,13,13]

In [ ]:
# Criação do objeto modelo
modelo  = grb.Model('transporte')


#criação de variáveis
x = {} # x é um dicionário
for i in I:
  for j in J:
    x[i,j] = modelo.addVar(name = f'x[{i},{j}]',vtype= grb.GRB.CONTINUOUS, lb=0)

#atualização do modelo
modelo.update()

#Restrições
for j in J:
  modelo.addConstr(grb.quicksum(x[i,j] for i in I) >= d[j-1])

#Restrições
for i in I:
  modelo.addConstr(grb.quicksum(x[i,j] for j in J) <= o[i-1])

#objetivo
Qx = grb.quicksum(c[i-1][j-1]*x[i,j] for i in I for j in J)

modelo.setObjective(Qx, sense=grb.GRB.MINIMIZE)

#execução
modelo.optimize()

#resultado
for i in I:
  for j in J:
    if x[i,j].x>0.001:
      print(f"x[{i},{j}]={x[i,j].x}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 11 rows, 30 columns and 60 nonzeros
Model fingerprint: 0x4d018647
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve time: 0.01s
Presolved: 11 rows, 30 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.900000e+01   0.000000e+00      0s
      12    9.4000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.02 seconds (0.00 work units)
Optimal objective  9.400000000e+02
x[1,2]=5.0
x[1,4]=6.0
x[2,5]=7.0
x[2,6]=6.0
x[3,5]=9.0
x[4,2]=3.0
x[4,6]=10.0
x[5,1]=5.0
x[5,3]=4.0
x[5,4]=4.0


Responda:


1.   [2 pontos] A solução ótima para a instância foi encontrada?
2.   [2 pontos] Qual o valor da função objetivo?
3. [2 pontos] Por onde e com qual valor o destino 5 é abastecido?
4. [2 pontos] Inclua um novo nó de demanda, ajuste a capacidade de oferta para conseguir atendê-lo e crie os custos de transporte para esse novo nó.
5. [2 pontos] Para essa nova instância, por onde e com qual valor o novo destino é abastecido?
   




#**Questão 2 (15 pontos)**

Considere que a empresa possa decidir se deseja abrir ou não um nó de oferta. Para tanto, caso o nó de oferta $i \in I$ seja aberto, será cobrado um custo fixo na função objetivo de $f_i$. Além disto, só será possível atender uma demanda a partir de $i$, caso o nó $i$ tenha sido aberto.

Sendo a variável $y_i$ uma variável binária que indica a abertura do nó $i$, temos a formulação a seguir:

\begin{eqnarray}
\label{of-tpl} \mbox{Minimize} \ & Q(x,y) = \sum\limits_{i \in I} \sum\limits_{j \in J}c_{ij} x_{ij}  + \sum_{i \in I}f_i y_i                 & \\
\nonumber \mbox{ Sujeito a:}
        &                                   & \\
\label{r1-tpl} &  \sum\limits_{i \in I} x_{ij} >= d_i   & \forall \quad j \in J \\
\label{r2-tpl} &  \sum\limits_{j \in J} x_{ij} <= o_i y_i   & \forall \quad i \in I \\
\label{r3-tpl} &  x_{ij}  \geq 0,  &\forall \quad i \in I, j \in J \\
\label{r4-tpl} &  y_{i}  \in \{0,1\},  &\forall \quad i \in I
\end{eqnarray}


In [ ]:
# Instância
I = [1,2,3,4,5]
J = [1,2,3,4,5]
c = [ [30, 20, 24, 18, 31],
      [12, 36, 30, 24,23],
      [8, 15, 25, 20, 17],
     [18, 5, 15, 30, 13],
     [7, 25, 15, 10, 27]]
f = [100,120, 90, 150, 130]
d = [5, 8, 4, 10, 16]
o = [20,20,20,20,20]

In [ ]:
# Criação do objeto modelo
modelo  = grb.Model('transporte-loc')


#criação de variáveis
x = {} # x é um dicionário
for i in I:
  for j in J:
    x[i,j] = modelo.addVar(name = f'x[{i},{j}]',vtype= grb.GRB.CONTINUOUS, lb=0)

# DEFINA A VARIÁVEL y DO TIPO grb.GRB.BINARY
y = {}
for i in I:
    y[i] = modelo.addVar(name = f'y[{i}]',vtype= grb.GRB.BINARY)

#atualização do modelo
modelo.update()

#Restrições
for j in J:
  modelo.addConstr(grb.quicksum(x[i,j] for i in I) >= d[j-1])

#Restrições
for i in I:
  modelo.addConstr(grb.quicksum(x[i,j] for j in J) <= o[i-1]*y[i]) # ALTERE A RESTRIÇÃO DE OFERTA

#objetivo
Qx = grb.quicksum(c[i-1][j-1]*x[i,j] for i in I for j in J)
Q = Qx + grb.quicksum(f[i-1] * y[i] for i in I)  # ALTERE A FUNÇÃO OBJETIVO

modelo.setObjective(Q, sense=grb.GRB.MINIMIZE)

#execução
modelo.optimize()

#resultado
for i in I:
  if y[i].x>0.001:
    print(f"y[{i}]={y[i].x}")
  for j in J:
    if x[i,j].x>0.001:
      print(f"\t x[{i},{j}]={x[i,j].x}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 30 columns and 55 nonzeros
Model fingerprint: 0xe6211bf7
Variable types: 25 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+01]
Presolve time: 0.00s
Presolved: 10 rows, 30 columns, 55 nonzeros
Variable types: 25 continuous, 5 integer (5 binary)
Found heuristic solution: objective 829.0000000

Root relaxation: objective 7.455000e+02, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  745.50000    0    2  829.00000  745.50000  10.

Faça:


1.   [2 pontos] Crie uma nova variável no modelo.
2.   [2 pontos] Altere a restrição de oferta.
3. [2 pontos] Altere a função objetivo.
4. [2 pontos] Execute o modelo e responda: Quais origens foram abertas?
5. [2 pontos] Qual é o valor da função objetivo?

#Questão 3

O *Product Owner - PO* de uma empresa de *software* necessita atender uma demanda urgente de um produto que já estava em produção. Para tanto, ele elaborou o *backlog* que contém 6 atividades que podem ser realizadas de forma independente e paralelas antes do teste de *software*.  Devido à elevada rotatividade de programadores, o PO deverá montar, novamente, seu time de desenvolvedores (Devs) para atender a demanda. O PO pode selecionar sua equipe dentro de um conjunto de 5 Devs que possuem seus próprios custos e disponibilidades conforme a Tabela 1. Adicionalmente, por terem competências diferentes, os Devs são capazes de desenvolver as atividades em tempos diferentes, conforme a Tabela 2. Considerando o objetivo de minimizar o custo de atendimento à demanda, a disponibilidade dos Devs e a necessidade de realizar todas as atividades, faça o que se pede.

Tabela 1:
\begin{array}{|c|} \hline
Dev & Disp. (h) & Custo (\$/h)  \\ \hline
D_1 & 6 & 7 \\
D_2 & 8 & 7  \\
D_3 & 12 & 12 \\
D_4 & 4 & 5  \\
D_5 & 9 & 5  \\
\hline
\end{array}

Tabela 2:
\begin{array}{|c|} \hline
Dev& A_1 & A_2 & A_3 & A_4 & A_5 & A_6\\\hline
D_1  & 2 & 2 & 3 & 5 & 1 & 2\\
D_2  & 3 & 2 & 2 & 4 & 1 & 3 \\
D_3 &1 &1 &1 &2 &1 &1\\
D_4 &3 &2 &1 &4 &1 &2\\
D_5 &4 &4 &4 &6 &2 &3 \\\hline
\end{array}

Formulação Matemática:


\begin{eqnarray}
\label{p-med-of} & \mbox{Minimize} \hspace{0.15cm}
        Q(x) = \sum_{d \in D}\sum_{a \in A}c_{ad}x_{ad} & &\\
        \nonumber & \mbox{Sujeito a:} &    \\
\label{p-med-r1} & \sum_{d \in D} x_{ad} = 1 & \forall  a \in A  \\
\label{p-med-r3} & \sum_{a \in A} x_{ad} \leq H_d & \forall  d \in D  \\
\label{p-med-r4} & x_{ad} \in \{0,1\} & \forall  d \in D, \forall  a \in A \\
\end{eqnarray}


Faça:


1.   [3 pontos] Implemente o problema utilizando o gurobipy.
2.   [3 pontos] Inclua os dados da instância e execute o problema.
3. [2 pontos] Responda: Quais atividades serão realizadas por cada dev?
4. [2 pontos] Qual é o valor da função objetivo?

In [23]:
#implemente aqui!!
DISP = [6,8,12,4,9] #DISPONIBILIDADE
Custo = [7,7,12,5,5] #CUSTO
Tempo = [ #TEMPO DE APLICAÇÃO
    [2,2,3,5,1,2],
    [3,2,2,4,1,3],
    [1,1,1,2,1,1],
    [3,2,1,4,1,2],
    [4,4,4,6,2,3]
]
D = [1,2,3,4,5] #DEVS
A = [1,2,3,4,5,6] #ATIVIDADES


modelo = grb.Model('Atividade_DEVS')

#DEFINIÇÃO DE VARIAVEIS
x = {}
for d in D:
    for a in A:
        x[d, a] = modelo.addVar(name=f'x[{d},{a}]', vtype=grb.GRB.BINARY)

#FUNÇÃO OBJETIVO
Q = grb.quicksum(Custo[D.index(d)] * Tempo[D.index(d)][A.index(a)] * x[d, a] for d in D for a in A)


#RESTRIÇÃO
for a in A:
    modelo.addConstr(grb.quicksum(x[d, a] for d in D) == 1)

for d in D:
    modelo.addConstr(grb.quicksum(x[d, a] * Tempo[D.index(d)][A.index(a)] for a in A) <= DISP[D.index(d)])


modelo.setObjective(Q, sense=grb.GRB.MINIMIZE)

modelo.optimize()

for d in D:
    for a in A:
        if x[d, a].x > 0.001:
            print(f"DEV: \t{d} ===> ATIVIDADE: {a}")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 11 rows, 30 columns and 60 nonzeros
Model fingerprint: 0xf91df546
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [5e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 100.0000000
Presolve removed 11 rows and 30 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 68 100 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.800000000000e+01, best bound 6.800000000000e+01, gap 0.0000%
DEV: 	3 ===> ATIVIDADE: 1
DEV: 	3 ===> ATIVI